In [1]:
import tracemalloc

from utils_v1 import *
import traceback


In [2]:
from icecream import ic

import numpy as np
from src.dataset import (
    MultipleDates, PAMultipleDates,
    MTMultipleDates, MSMultipleDates, PIMultipleDates
)

# from src.dataset_legacy import MTDeforestationTime
from src import training_generator as generator
from src.patchesHandler import PatchesHandler, PatchesHandlerMultipleDates, PatchesHandlerEvidential2

from src.manager.evidential import ManagerEvidential
from src.manager.multioutput import ManagerMCDropout, ManagerSingleRun, ManagerEvidential2
from mpl_toolkits.axes_grid1 import make_axes_locatable

import cv2
from src import metrics as _metric
import json
import pickle 
import src.manager.evidential as evidential
from src.Logger import Logger
import pdb


Tensorflow ver. 2.6.0


In [3]:

with open('config/config.json', 'r') as f:
    config = json.load(f)
    
print("Mode: {}. Site: {}. Training: {}. Training date: {}".format(
    config['mode'], config['site'], config['training'], config['training_date']))
print(config)


Mode: evidential2. Site: PA. Training: True. Training date: current
{'training': True, 'inferring': True, 'site': 'PA', 'training_date': 'current', 'mode': 'evidential2', 'training_times': 1, 'uncertainty_method': 'pred_entropy', 'loadInference': False, 'removePolygons': True, 'plotLandsat': False, 'plotPOI': False, 'save_probabilities': False, 'addPastDeforestationInput': True, 'classes_mode': True, 'learning_rate': 0.0001, 'use_cloud_mask': True}


In [4]:

if config['site'] == 'PA':
	if config['training_date'] == 'earlier':
		dates = [2017, 2018]
	else:
		dates = [2018, 2019]
	datasetClass = PAMultipleDates
elif config['site'] == 'MT':
	if config['training_date'] == 'earlier':
		dates = [2018, 2019]
	else:
		dates = [2019, 2020]
	datasetClass = MTMultipleDates
elif config['site'] == 'MS':
	if config['training_date'] == 'earlier':
		dates = [2018, 2019]
	else:
		dates = [2019, 2020]
	datasetClass = MSMultipleDates
elif config['site'] == 'PI':
	if config['training_date'] == 'earlier':
		dates = [2018, 2019]
	else:
		dates = [2019, 2020]
	datasetClass = PIMultipleDates
else:
	raise ValueError('Invalid site')

dataset = datasetClass(config = config,
			   dates = dates, 
	 	addPastDeforestationInput = config['addPastDeforestationInput'],
		borderBuffer = 2)	

if config['mode'] == 'ensemble':
	if config['training'] == True:
		config['mode'] = 'single_run'
	else:
		sys.exit("Ensure that training is True for ensemble mode. Use infer_ensemble.ipynb for inference.")

if config['training'] == False:
	if config['site'] == 'PA':
		if config['mode'] == 'mcd' or config['mode'] == 'single_run': # MCD will do 10 inference on each of the 10 repetitions = 100 runs
			if config['training_date'] == 'current': exp = 0
			elif config['training_date'] == 'earlier': exp = 1
				
		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 8; repetition_id = 0
			elif config['training_date'] == 'earlier': exp = 9; repetition_id = 0

	else:

		if config['mode'] == 'mcd' or config['mode'] == 'single_run':
			if config['training_date'] == 'current': exp = 2
			elif config['training_date'] == 'earlier': exp = 3
		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 10; repetition_id = 0
			elif config['training_date'] == 'earlier': exp = 11; repetition_id = 0

else:
	exp = 8
ic(dates)


if issubclass(type(dataset), MultipleDates):
	patchesHandler = PatchesHandlerMultipleDates(
		dataset)
elif config['mode'] == 'evidential2':
	patchesHandler = PatchesHandlerEvidential2()
else:
	patchesHandler = PatchesHandler()



ic| list(self.date_ids): [0]
ic| self.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
ic| dates: [2018, 2019]
ic| self.input_image_shape: 21
ic| self.dataset.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]


In [5]:


if config['mode'] == 'evidential2':
    config["inference_times"] = 1
    manager_class = ManagerEvidential2
    config['uncertainty_method'] = "pred_entropy"
    config['classes_mode'] = True
    
if config['mode'] == 'mcd':
    config["inference_times"] = 10
    manager_class = ManagerMCDropout
    config['uncertainty_method'] = "pred_entropy"
    config['classes_mode'] = False

elif config['mode'] == 'single_run':
    config["inference_times"] = 1
    manager_class = ManagerSingleRun
    config['uncertainty_method'] = "pred_entropy_single"
    config['classes_mode'] = False



In [6]:
exp

8

In [7]:
logger = Logger()


In [8]:
if config['training'] == True:
    

    manager = manager_class(config, dataset, patchesHandler, logger)
    print("manager.config['dropout_training']", manager.config['dropout_training'])
    manager.defineExperiment(exp) # fixed
    
    manager.setExperimentPath()
    manager.createLogFolders()

    manager.loadDataset()

    # %%
    if config["training"] == True:
        manager.run()


manager.config['dropout_training'] False
D:/Jorge/datasets/deforestation/PA/deforestation_past_years.tif
Loaded deforestation past years


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([132185553,   3150503,   4571354,    864429,    903608,   1299506,
                                                                      2477971,   1561686,   1459696,    969060,    644976,   2209014,
                                                                      2100204,   5257014,   3514616], dtype=int64))


Label where deforestation past years is actual date (2019) = 1


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([132185553,   3150503,   4571354,    864429,    903608,   1299506,
                                                                      2477971,   1561686,   1459696,    969060,    644976,   2209014,
                                                                      2100204,   5257014,   3514616], dtype=int64))


Past deforestation different from 0 (no deforestation)


...

training samples:  (3753, 3) validation samples:  (1032, 3)
time:  0


c:\Users\jchamorro\Anaconda3\envs\tf2\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_enc_net (InputLayer)      [(None, 128, 128, 21 0                                            
__________________________________________________________________________________________________
res1_net1 (Conv2D)              (None, 128, 128, 16) 3040        input_enc_net[0][0]              
__________________________________________________________________________________________________
drop_net1 (SpatialDropout2D)    (None, 128, 128, 16) 0           res1_net1[0][0]                  
__________________________________________________________________________________________________
res2_net1 (Conv2D)              (None, 128, 128, 16) 2320        drop_net1[0][0]                  
______________________________________________________________________________________________

c:\Users\jchamorro\Anaconda3\envs\tf2\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


351/351 [==============================] - 32s 91ms/step - loss: 0.2679 - accuracy: 0.4556 - val_loss: 0.2703 - val_accuracy: 0.5151

Epoch 00001: val_loss improved from 0.30724 to 0.27027, saving model to D:/Jorge/datasets/deforestation/experiments/PA/exp8/models\resunet_0.h5
New best val loss. Val loss: 0.27027. Early stop count: 0
Epoch: 3
Anneling Coeficient [0.2]
351/351 [==============================] - 32s 91ms/step - loss: 0.2561 - accuracy: 0.4560 - val_loss: 0.2684 - val_accuracy: 0.5107

Epoch 00001: val_loss improved from 0.27027 to 0.26841, saving model to D:/Jorge/datasets/deforestation/experiments/PA/exp8/models\resunet_0.h5
New best val loss. Val loss: 0.26841. Early stop count: 0
Epoch: 4
Anneling Coeficient [0.3]
351/351 [==============================] - 32s 90ms/step - loss: 0.2517 - accuracy: 0.4558 - val_loss: 0.2658 - val_accuracy: 0.5111

Epoch 00001: val_loss improved from 0.26841 to 0.26584, saving model to D:/Jorge/datasets/deforestation/experiments/PA/exp8/

In [9]:



if config['site'] == 'PA':
	dates = [2018, 2019]
	datasetClass = PAMultipleDates
elif config['site'] == 'MT':
	dates = [2019, 2020]
	datasetClass = MTMultipleDates
elif config['site'] == 'MS':
	dates = [2019, 2020]
	datasetClass = MSMultipleDates
elif config['site'] == 'PI':
	dates = [2019, 2020]
	datasetClass = PIMultipleDates
else:
	raise ValueError('Invalid site')

dataset = datasetClass(config = config,
			   dates = dates, 
	 	addPastDeforestationInput = config['addPastDeforestationInput'],
		borderBuffer = 2)


ic(dates)

if issubclass(type(dataset), MultipleDates):
	patchesHandler = PatchesHandlerMultipleDates(
		dataset)
elif config['mode'] == 'evidential2':
	patchesHandler = PatchesHandlerEvidential2()
else:
	patchesHandler = PatchesHandler()


ic| list(self.date_ids): [0]
ic| self.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
ic| dates: [2018, 2019]
ic| self.input_image_shape: 21
ic| self.dataset.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]


In [10]:
manager_class

src.manager.multioutput.ManagerEvidential2

In [11]:
exp

8

In [12]:
repetition_ids = manager_class(config, dataset, patchesHandler, logger).get_repetition_ids(exp)  
repetition_n = len(repetition_ids)
print("repetition_ids {}, repetition_n {}".format(repetition_ids, repetition_n))


repetition_ids [0], repetition_n 1


In [13]:
if config['inferring'] == True:
    idx = 0
    results = []
    error_count = 0

    # while idx < len(repetition_n):
    # for experiment_value, exp_id in zip(experiment_values, exp_ids):
    for idx in range(repetition_n): # 10 repetitions


        print("Beginning run number {}".format(idx))
        logger = Logger()
        manager = manager_class(config, dataset, patchesHandler, logger)
        print("manager.config", manager.config)

        manager.defineExperiment(exp) # fixed
        manager.defineRepetitionId(idx) # varying from 0 to 10
        manager.setExperimentPath()
        manager.createLogFolders()

        manager.loadDataset()

        # %%
        result = manager.run_predictor()
        print("Result idx {}: {}".format(idx, result))
        results.append(result)


        with open(manager.default_log_name, 'wb') as f:
            pickle.dump(results, f)
        print("Grid execution idx: {}".format(idx))
        # idx += 1
    

Beginning run number 0
manager.config {'training': True, 'inferring': True, 'site': 'PA', 'training_date': 'current', 'mode': 'evidential2', 'training_times': 1, 'uncertainty_method': 'pred_entropy', 'loadInference': False, 'removePolygons': True, 'plotLandsat': False, 'plotPOI': False, 'save_probabilities': False, 'addPastDeforestationInput': True, 'classes_mode': True, 'learning_rate': 0.0001, 'use_cloud_mask': True, 'inference_times': 1, 'dropout_training': False}


D:/Jorge/datasets/deforestation/PA/deforestation_past_years.tif
Loaded deforestation past years


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([132185553,   3150503,   4571354,    864429,    903608,   1299506,
                                                                      2477971,   1561686,   1459696,    969060,    644976,   2209014,
                                                                      2100204,   5257014,   3514616], dtype=int64))


Label where deforestation past years is actual date (2019) = 1


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([132185553,   3150503,   4571354,    864429,    903608,   1299506,
                                                                      2477971,   1561686,   1459696,    969060,    644976,   2209014,
                                                                      2100204,   5257014,   3514616], dtype=int64))


...

[0.86399145 0.95444234 0.7098381  0.03000392 0.58204377 0.2717464
 0.10051083]
threshold:  0.4191759345647506
threshold 0.4191759345647506


ic| label_current_deforestation_test_classified_incorrect.shape: (1624523,)
    predicted_test_classified_incorrect.shape: (1624523,)
ic| TP_H + FN_H + FP_H + TN_H: 1624523
    len(label_mask_current_deforestation_test): 54151420
ic| TP_H + FN_H + FP_H + TN_H: 1624523
    len(label_mask_current_deforestation_test): 54151420


[0.86397872 0.95444451 0.70987345 0.02999964 0.58200625 0.27164594
 0.10051604]
threshold 0.4191759345647506
(54151420,) (54151420,)


ic| self.m_optimal: {'AA': array([0.02999964]),
                     'UEO': array([0.10051604]),
                     'f1': 80.31,
                     'f1_H': array([0.37040761]),
                     'f1_L': array([0.90696129]),
                     'precision_H': array([0.58200625]),
                     'precision_L': array([0.86397872]),
                     'recall_H': array([0.27164594]),
                     'recall_L': array([0.95444451]),
                     'recall_Ltotal': array([0.70987345])}
ic| self.m_audited_optimal: {'f1': array([0.92990841]),
                             'precision': array([0.89631513]),
                             'recall': array([0.96611785])}


cm_audited [[53333633    82208]
 [   24923   710656]]
[0.89631513 0.96611785]
Result idx 0: {'uncertainty_result': {'metrics': {'precision_L': array([0.86397872]), 'recall_L': array([0.95444451]), 'recall_Ltotal': array([0.70987345]), 'AA': array([0.02999964]), 'precision_H': array([0.58200625]), 'recall_H': array([0.27164594]), 'UEO': array([0.10051604]), 'f1_L': array([0.90696129]), 'f1_H': array([0.37040761]), 'f1': 80.31}, 'metrics_audited': {'precision': array([0.89631513]), 'recall': array([0.96611785]), 'f1': array([0.92990841])}, 'exp': 8}}
Grid execution idx: 0


In [14]:
error_count

0

In [15]:
print(results)

[{'uncertainty_result': {'metrics': {'precision_L': array([0.86397872]), 'recall_L': array([0.95444451]), 'recall_Ltotal': array([0.70987345]), 'AA': array([0.02999964]), 'precision_H': array([0.58200625]), 'recall_H': array([0.27164594]), 'UEO': array([0.10051604]), 'f1_L': array([0.90696129]), 'f1_H': array([0.37040761]), 'f1': 80.31}, 'metrics_audited': {'precision': array([0.89631513]), 'recall': array([0.96611785]), 'f1': array([0.92990841])}, 'exp': 8}}]


In [16]:
print(len(results))

1


In [17]:
results[0]

{'uncertainty_result': {'metrics': {'precision_L': array([0.86397872]),
   'recall_L': array([0.95444451]),
   'recall_Ltotal': array([0.70987345]),
   'AA': array([0.02999964]),
   'precision_H': array([0.58200625]),
   'recall_H': array([0.27164594]),
   'UEO': array([0.10051604]),
   'f1_L': array([0.90696129]),
   'f1_H': array([0.37040761]),
   'f1': 80.31},
  'metrics_audited': {'precision': array([0.89631513]),
   'recall': array([0.96611785]),
   'f1': array([0.92990841])},
  'exp': 8}}

In [18]:
manager.f1

80.31

In [19]:
print(manager.f1)

80.31
